In [396]:
"""
# Remove all conda packages
!find /opt/conda \( -name "cudf*" -o -name "libcudf*" -o -name "cuml*" -o -name "libcuml*" \
                   -o -name "cugraph*" -o -name "libcugraph*" -o -name "raft*" -o -name "libraft*" \
                   -o -name "pylibraft*" -o -name "libkvikio*" -o -name "*dask*" -o -name "rmm*"\
                   -o -name "librmm*" \) -exec rm -rf {} \; 2>/dev/null

# pip uninstall, just incase there are packages lying around
!pip uninstall -q cudf cuml dask xgboost dask-cudf cuml cugraph cupy cupy-cuda12x --y


!pip install \
    -q \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.2.* \
    dask-cudf-cu12==24.2.* \
    cuml-cu12==24.2.* \
    cugraph-cu12==24.2.*
    
"""

'\n# Remove all conda packages\n!find /opt/conda \\( -name "cudf*" -o -name "libcudf*" -o -name "cuml*" -o -name "libcuml*"                    -o -name "cugraph*" -o -name "libcugraph*" -o -name "raft*" -o -name "libraft*"                    -o -name "pylibraft*" -o -name "libkvikio*" -o -name "*dask*" -o -name "rmm*"                   -o -name "librmm*" \\) -exec rm -rf {} \\; 2>/dev/null\n\n# pip uninstall, just incase there are packages lying around\n!pip uninstall -q cudf cuml dask xgboost dask-cudf cuml cugraph cupy cupy-cuda12x --y\n\n\n!pip install     -q     --extra-index-url=https://pypi.nvidia.com     cudf-cu12==24.2.*     dask-cudf-cu12==24.2.*     cuml-cu12==24.2.*     cugraph-cu12==24.2.*\n    \n'

In [397]:
!pip install -q jupyter-black
!pip install -q --upgrade xgboost[dask]

In [398]:
import cudf
%load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [399]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns

from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
from sklearn.model_selection import GroupKFold


In [400]:
plt.style.use("fivethirtyeight")
mypal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
print(mypal)

['#008fd5', '#fc4f30', '#e5ae38', '#6d904f', '#8b8b8b', '#810f7c']


In [401]:
!ls -GFlash ../input/march-machine-learning-mania-2024/

total 144M
   0 drwxr-xr-x 2 nobody    0 Mar 18 20:22 ./
4.0K drwxr-xr-x 5 root   4.0K Mar 21 02:02 ../
4.0K -rw-r--r-- 1 nobody 1.4K Mar 18 20:22 2024_tourney_seeds.csv
 12K -rw-r--r-- 1 nobody 9.1K Mar 18 20:22 Cities.csv
4.0K -rw-r--r-- 1 nobody 1.7K Mar 18 20:22 Conferences.csv
168K -rw-r--r-- 1 nobody 168K Mar 18 20:22 MConferenceTourneyGames.csv
2.5M -rw-r--r-- 1 nobody 2.5M Mar 18 20:22 MGameCities.csv
111M -rw-r--r-- 1 nobody 111M Mar 18 20:22 MMasseyOrdinals_thruSeason2024_day128.csv
 72K -rw-r--r-- 1 nobody  72K Mar 18 20:22 MNCAATourneyCompactResults.csv
132K -rw-r--r-- 1 nobody 129K Mar 18 20:22 MNCAATourneyDetailedResults.csv
 16K -rw-r--r-- 1 nobody  16K Mar 18 20:22 MNCAATourneySeedRoundSlots.csv
 40K -rw-r--r-- 1 nobody  38K Mar 18 20:22 MNCAATourneySeeds.csv
 52K -rw-r--r-- 1 nobody  50K Mar 18 20:22 MNCAATourneySlots.csv
5.3M -rw-r--r-- 1 nobody 5.3M Mar 18 20:22 MRegularSeasonCompactResults.csv
 11M -rw-r--r-- 1 nobody  11M Mar 18 20:22 MRegularSeasonDetailedResults.

In [402]:
#Read in data
DATA_PATH = "../input/march-machine-learning-mania-2024/"
df_seeds = pd.concat([pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv").assign(League="Men"),
                     pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv").assign(League="Women")]).reset_index(drop=True)

df_season_results = pd.concat([pd.read_csv(DATA_PATH + "MRegularSeasonDetailedResults.csv").assign(League="Men"),
                     pd.read_csv(DATA_PATH + "WRegularSeasonDetailedResults.csv").assign(League="Women")]).reset_index(drop=True)

df_tourney_results = pd.concat([pd.read_csv(DATA_PATH + "MNCAATourneyDetailedResults.csv").assign(League="Men"),
                     pd.read_csv(DATA_PATH + "WNCAATourneyDetailedResults.csv").assign(League="Women")]).reset_index(drop=True)

df_seeds

,Season,Seed,TeamID,League
0,1985,W01,1207,Men
1,1985,W02,1210,Men
2,1985,W03,1228,Men
3,1985,W04,1260,Men
4,1985,W05,1374,Men
...,...,...,...,...
4229,2024,Z12b,3435,Women
4230,2024,Z13,3267,Women
4231,2024,Z14,3238,Women
4232,2024,Z15,3263,Women


In [403]:
df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,10,1104,68,1328,62,N,0,27,58,...,16,22,10,22,8,18,9,2,20,Men
1,2003,10,1272,70,1393,63,N,0,26,62,...,9,20,20,25,7,12,8,6,16,Men
2,2003,11,1266,73,1437,61,N,0,24,58,...,14,23,31,22,9,12,2,5,23,Men
3,2003,11,1296,56,1457,50,N,0,18,38,...,8,15,17,20,9,19,4,3,23,Men
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189433,2024,131,3465,75,3372,74,N,0,25,64,...,10,16,9,30,22,12,8,5,15,Women
189434,2024,132,3179,76,3283,75,N,0,28,52,...,10,13,18,15,14,14,9,3,17,Women
189435,2024,132,3180,68,3392,60,N,0,23,46,...,17,21,21,17,7,18,9,2,14,Women
189436,2024,132,3221,61,3131,55,H,0,18,52,...,5,7,6,27,10,17,6,4,21,Women


In [404]:
df_tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,134,1421,92,1411,84,N,1,32,69,...,14,31,17,28,16,15,5,0,22,Men
1,2003,136,1112,80,1436,51,N,0,31,66,...,7,7,8,26,12,17,10,3,15,Men
2,2003,136,1113,84,1272,71,N,0,31,59,...,14,21,20,22,11,12,2,5,18,Men
3,2003,136,1141,79,1166,73,N,0,29,53,...,12,17,14,17,20,21,6,6,21,Men
4,2003,136,1143,76,1301,74,N,1,27,64,...,15,20,10,26,16,14,5,8,19,Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,2023,147,3376,86,3268,75,H,0,32,70,...,10,15,6,17,15,12,7,9,26,Women
2138,2023,147,3439,84,3326,74,N,0,26,54,...,15,17,4,17,11,10,6,1,25,Women
2139,2023,151,3234,77,3376,73,N,0,28,57,...,9,13,24,22,9,15,6,5,20,Women
2140,2023,151,3261,79,3439,72,N,0,33,70,...,17,18,13,23,8,15,7,8,15,Women


In [405]:
winning  = (
df_season_results[["Season", "League", "WTeamID", "LTeamID", "DayNum", "WScore","LScore", "WTO", "LTO", "WFGM3", "WFGA3", "LFGM3", "LFGA3", "WAst", "LAst", "WStl", "WBlk", "LStl", "LBlk", "WOR", "LOR"]]).assign(GameResult="W").rename(columns={"WTeamID":"TeamID", "LTeamID":"OppTeamID", "WScore":"TeamScore", "LScore":"OppScore", "WTO": "TeamTO", "LTO": "OppTO", "WFGM3": "TeamFGM3","WFGA3":"TeamFGA3", "LFGM3":"OppFGM3", "LFGA3":"OppFGA3", "WAst":"TeamAst", "LAst":"OppAst", "WStl":"TeamStl", "WBlk":"TeamBlk", "LStl":"OppStl", "LBlk":"OppBlk", "WOR":"TeamOR", "LOR":"OppOR"})

In [406]:
winning.head()

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,OppFGA3,TeamAst,OppAst,TeamStl,TeamBlk,OppStl,OppBlk,TeamOR,OppOR,GameResult
0,2003,Men,1104,1328,10,68,62,23,18,3,...,10,13,8,7,1,9,2,14,10,W
1,2003,Men,1272,1393,10,70,63,13,12,8,...,24,16,7,4,4,8,6,15,20,W
2,2003,Men,1266,1437,11,73,61,10,12,8,...,26,15,9,5,2,2,5,17,31,W
3,2003,Men,1296,1457,11,56,50,12,19,3,...,22,11,9,14,2,4,3,6,17,W
4,2003,Men,1400,1208,11,77,71,14,10,6,...,16,12,12,4,4,7,1,17,21,W


In [407]:
losing = (
df_season_results[["Season", "League", "WTeamID", "DayNum", "WScore","LScore", "WTO", "LTO", "WFGM3", "WFGA3", "LFGM3", "LFGA3", "WAst", "LAst", "WStl", "WBlk", "LStl", "LBlk", "WOR", "LOR"]]).assign(GameResult="L").rename(columns={"LTeamID":"TeamID", "WTeamID":"OppTeamID","LScore":"TeamScore", "WScore":"OppScore", "LTO": "TeamTO", "WTO": "OppTO", "LFGM3": "TeamFGM3","LFGA3":"TeamFGA3", "WFGM3":"OppFGM3", "WFGA3":"OppFGA3", "LAst":"TeamAst", "WAst":"OppAst", "LStl":"TeamStl", "LBlk":"TeamBlk", "WStl":"OppStl", "WBlk":"OppBlk", "LOR":"TeamOR", "WOR":"OppOR"})

In [408]:
df_team_season_results = pd.concat(
[winning, losing]).reset_index(drop = True)

In [409]:
df_team_season_results

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,OppFGA3,TeamAst,OppAst,TeamStl,TeamBlk,OppStl,OppBlk,TeamOR,OppOR,GameResult
0,2003,Men,1104.0,1328,10,68,62,23,18,3,...,10,13,8,7,1,9,2,14,10,W
1,2003,Men,1272.0,1393,10,70,63,13,12,8,...,24,16,7,4,4,8,6,15,20,W
2,2003,Men,1266.0,1437,11,73,61,10,12,8,...,26,15,9,5,2,2,5,17,31,W
3,2003,Men,1296.0,1457,11,56,50,12,19,3,...,22,11,9,14,2,4,3,6,17,W
4,2003,Men,1400.0,1208,11,77,71,14,10,6,...,16,12,12,4,4,7,1,17,21,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378871,2024,Women,NaN,3465,131,74,75,12,11,10,...,24,22,13,8,5,7,4,9,7,L
378872,2024,Women,NaN,3179,132,75,76,14,19,3,...,17,14,16,9,3,9,3,18,7,L
378873,2024,Women,NaN,3180,132,60,68,18,21,5,...,19,7,17,9,2,9,4,21,6,L
378874,2024,Women,NaN,3221,132,55,61,17,10,8,...,26,10,10,6,4,6,3,6,12,L


In [410]:
df_team_season_results["point_differential"] = df_team_season_results["TeamScore"] - df_team_season_results["OppScore"]
df_team_season_results["turnover_differential"] = df_team_season_results["TeamTO"] - df_team_season_results["OppTO"]
df_team_season_results["3pt_differential"] = (df_team_season_results["TeamFGM3"] / df_team_season_results["TeamFGA3"]) - (df_team_season_results["OppFGM3"] / df_team_season_results["OppFGA3"])
df_team_season_results["assist_differential"] = df_team_season_results["TeamAst"] - df_team_season_results["OppAst"]
df_team_season_results["defensive_differential"] = (df_team_season_results["TeamStl"] + df_team_season_results["TeamBlk"]) - (df_team_season_results["OppStl"] + df_team_season_results["OppBlk"])
df_team_season_results["offensivereb_differential"] = df_team_season_results["TeamOR"] - df_team_season_results["OppOR"]

In [411]:
df_team_season_results["Win"] = (df_team_season_results["GameResult"] == "W").astype("int")

In [412]:
df_team_season_results.sample(10, random_state=529)

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,TeamOR,OppOR,GameResult,point_differential,turnover_differential,3pt_differential,assist_differential,defensive_differential,offensivereb_differential,Win
155301,2018,Women,3371.0,3240,27,73,54,19,21,5,...,13,12,W,19,-2,0.000000,2,6,1,1
26567,2008,Men,1345.0,1336,79,64,42,9,13,11,...,7,3,W,22,-4,-0.115556,8,14,4,1
280200,2020,Men,NaN,1329,82,62,73,9,13,7,...,13,7,L,-11,-4,-0.141053,-7,-4,6,0
234745,2012,Men,NaN,1301,25,67,82,14,11,7,...,10,19,L,-15,3,-0.136364,-3,-5,-9,0
208804,2007,Men,NaN,1265,27,78,89,12,12,8,...,13,3,L,-11,0,-0.081871,-8,-4,10,0
263445,2017,Men,NaN,1230,75,77,82,12,7,9,...,15,6,L,-5,5,0.027174,-1,-6,9,0
321929,2013,Women,NaN,3321,104,63,70,14,16,8,...,12,6,L,-7,-2,0.077922,-6,-1,6,0
140849,2015,Women,3251.0,3219,61,64,47,24,16,2,...,13,11,W,17,8,-0.085714,4,-4,2,1
162935,2019,Women,3388.0,3339,87,113,85,20,14,10,...,14,8,W,28,6,0.214286,6,3,6,1
133604,2014,Women,3281.0,3387,4,74,59,13,14,5,...,11,16,W,15,-1,-0.065789,2,8,-5,1


In [413]:
team_season_agg = (df_team_season_results.groupby(["Season", "TeamID", "League"])).agg(
AvgScoreDiff = ("point_differential", "mean"),
MedianScoreDiff = ("point_differential", "median"),
MinScoreDiff = ("point_differential", "min"),
MaxScoreDiff = ("point_differential", "max"),
AvgTurnoverDiff = ("turnover_differential", "mean"),
Avg3ptDiff = ("3pt_differential", "mean"),
AvgAssistDiff = ("assist_differential", "mean"),
AvgDefensiveDiff = ("defensive_differential", "mean"),
AvgOffensiveRebDiff = ("offensivereb_differential", "mean"),
Wins = ("Win", "sum"),
Losses = ("GameResult", lambda x: (x == "L").sum()),
WinPercentage = ("Win", "mean"),
).reset_index()


In [414]:
team_season_agg

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,AvgTurnoverDiff,Avg3ptDiff,AvgAssistDiff,AvgDefensiveDiff,AvgOffensiveRebDiff,Wins,Losses,WinPercentage
0,2003,1102.0,Men,15.583333,13.5,4,38,-3.083333,0.135258,9.250000,3.333333,-6.750000,12,0,1.0
1,2003,1103.0,Men,9.384615,7.0,1,33,-3.461538,0.055263,4.000000,0.230769,-4.000000,13,0,1.0
2,2003,1104.0,Men,13.176471,9.0,5,28,-2.058824,0.041822,3.176471,3.705882,2.294118,17,0,1.0
3,2003,1105.0,Men,13.000000,8.0,1,34,-4.000000,0.147515,3.428571,2.142857,1.428571,7,0,1.0
4,2003,1106.0,Men,10.384615,9.0,1,26,1.769231,0.093951,3.769231,2.000000,1.307692,13,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12823,2024,3474.0,Women,9.000000,7.5,3,18,-2.500000,0.049784,-5.250000,3.500000,-1.750000,4,0,1.0
12824,2024,3475.0,Women,15.863636,13.0,3,38,1.409091,0.127380,5.318182,-1.454545,0.681818,22,0,1.0
12825,2024,3476.0,Women,11.500000,9.5,4,23,-2.000000,0.250782,6.500000,-4.000000,-2.500000,4,0,1.0
12826,2024,3477.0,Women,8.846154,9.0,1,24,-2.230769,0.110243,-0.153846,1.076923,-1.307692,13,0,1.0


In [415]:
df_seeds

,Season,Seed,TeamID,League
0,1985,W01,1207,Men
1,1985,W02,1210,Men
2,1985,W03,1228,Men
3,1985,W04,1260,Men
4,1985,W05,1374,Men
...,...,...,...,...
4229,2024,Z12b,3435,Women
4230,2024,Z13,3267,Women
4231,2024,Z14,3238,Women
4232,2024,Z15,3263,Women


In [416]:
df_seeds["TourneySeed"] = (df_seeds["Seed"].str.replace("a", "").str.replace("b","").str[1:].astype("int"))


In [417]:
df_seeds

,Season,Seed,TeamID,League,TourneySeed
0,1985,W01,1207,Men,1
1,1985,W02,1210,Men,2
2,1985,W03,1228,Men,3
3,1985,W04,1260,Men,4
4,1985,W05,1374,Men,5
...,...,...,...,...,...
4229,2024,Z12b,3435,Women,12
4230,2024,Z13,3267,Women,13
4231,2024,Z14,3238,Women,14
4232,2024,Z15,3263,Women,15


In [418]:
team_season_agg = team_season_agg.merge(df_seeds, on=["Season", "TeamID", "League"], how="left")

In [419]:
team_season_agg

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,AvgTurnoverDiff,Avg3ptDiff,AvgAssistDiff,AvgDefensiveDiff,AvgOffensiveRebDiff,Wins,Losses,WinPercentage,Seed,TourneySeed
0,2003,1102.0,Men,15.583333,13.5,4,38,-3.083333,0.135258,9.250000,3.333333,-6.750000,12,0,1.0,NaN,NaN
1,2003,1103.0,Men,9.384615,7.0,1,33,-3.461538,0.055263,4.000000,0.230769,-4.000000,13,0,1.0,NaN,NaN
2,2003,1104.0,Men,13.176471,9.0,5,28,-2.058824,0.041822,3.176471,3.705882,2.294118,17,0,1.0,Y10,10.0
3,2003,1105.0,Men,13.000000,8.0,1,34,-4.000000,0.147515,3.428571,2.142857,1.428571,7,0,1.0,NaN,NaN
4,2003,1106.0,Men,10.384615,9.0,1,26,1.769231,0.093951,3.769231,2.000000,1.307692,13,0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12823,2024,3474.0,Women,9.000000,7.5,3,18,-2.500000,0.049784,-5.250000,3.500000,-1.750000,4,0,1.0,NaN,NaN
12824,2024,3475.0,Women,15.863636,13.0,3,38,1.409091,0.127380,5.318182,-1.454545,0.681818,22,0,1.0,NaN,NaN
12825,2024,3476.0,Women,11.500000,9.5,4,23,-2.000000,0.250782,6.500000,-4.000000,-2.500000,4,0,1.0,NaN,NaN
12826,2024,3477.0,Women,8.846154,9.0,1,24,-2.230769,0.110243,-0.153846,1.076923,-1.307692,13,0,1.0,NaN,NaN


In [420]:
df_tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,League
0,2003,134,1421,92,1411,84,N,1,32,69,...,14,31,17,28,16,15,5,0,22,Men
1,2003,136,1112,80,1436,51,N,0,31,66,...,7,7,8,26,12,17,10,3,15,Men
2,2003,136,1113,84,1272,71,N,0,31,59,...,14,21,20,22,11,12,2,5,18,Men
3,2003,136,1141,79,1166,73,N,0,29,53,...,12,17,14,17,20,21,6,6,21,Men
4,2003,136,1143,76,1301,74,N,1,27,64,...,15,20,10,26,16,14,5,8,19,Men
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,2023,147,3376,86,3268,75,H,0,32,70,...,10,15,6,17,15,12,7,9,26,Women
2138,2023,147,3439,84,3326,74,N,0,26,54,...,15,17,4,17,11,10,6,1,25,Women
2139,2023,151,3234,77,3376,73,N,0,28,57,...,9,13,24,22,9,15,6,5,20,Women
2140,2023,151,3261,79,3439,72,N,0,33,70,...,17,18,13,23,8,15,7,8,15,Women


In [421]:
winning  = (
df_tourney_results[["Season", "League", "WTeamID","LTeamID", "DayNum", "WScore","LScore", "WTO", "LTO", "WFGM3", "WFGA3", "LFGM3", "LFGA3", "WAst", "LAst", "WStl", "WBlk", "LStl", "LBlk", "WOR", "LOR"]]).assign(GameResult="W").rename(columns={"WTeamID":"TeamID", "LTeamID" : "OppTeamID", "WScore":"TeamScore", "LScore":"OppScore", "WTO": "TeamTO", "LTO": "OppTO", "WFGM3": "TeamFGM3","WFGA3":"TeamFGA3", "LFGM3":"OppFGM3", "LFGA3":"OppFGA3", "WAst":"TeamAst", "LAst":"OppAst", "WStl":"TeamStl", "WBlk":"TeamBlk", "LStl":"OppStl", "LBlk":"OppBlk", "WOR":"TeamOR", "LOR":"OppOR"})
losing = (
df_tourney_results[["Season", "League", "LTeamID", "WTeamID", "DayNum", "WScore","LScore", "WTO", "LTO", "WFGM3", "WFGA3", "LFGM3", "LFGA3", "WAst", "LAst", "WStl", "WBlk", "LStl", "LBlk", "WOR", "LOR"]]).assign(GameResult="L").rename(columns={"LTeamID":"TeamID", "WTeamID":"OppTeamID", "LScore":"TeamScore", "WScore":"OppScore", "LTO": "TeamTO", "WTO": "OppTO", "LFGM3": "TeamFGM3","LFGA3":"TeamFGA3", "WFGM3":"OppFGM3", "WFGA3":"OppFGA3", "LAst":"TeamAst", "WAst":"OppAst", "LStl":"TeamStl", "LBlk":"TeamBlk", "WStl":"OppStl", "WBlk":"OppBlk", "LOR":"TeamOR", "WOR":"OppOR"})
df_team_tourney_results = pd.concat(
[winning, losing]).reset_index(drop = True)

df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype("int")

In [422]:
df_team_tourney_results.head()

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,TeamAst,OppAst,TeamStl,TeamBlk,OppStl,OppBlk,TeamOR,OppOR,GameResult,Win
0,2003,Men,1421,1411,134,92,84,12,15,11,...,17,16,5,3,5,0,14,17,W,1
1,2003,Men,1112,1436,136,80,51,16,17,7,...,22,12,10,7,10,3,11,8,W,1
2,2003,Men,1113,1272,136,84,71,9,12,6,...,18,11,7,4,2,5,10,20,W,1
3,2003,Men,1141,1166,136,79,73,18,21,3,...,15,20,13,1,6,6,11,14,W,1
4,2003,Men,1143,1301,136,76,74,13,14,7,...,17,16,8,2,5,8,18,10,W,1


In [423]:
df_historic_tourney_features = df_team_tourney_results.merge(team_season_agg[["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "AvgTurnoverDiff", "Avg3ptDiff", "AvgAssistDiff", "AvgDefensiveDiff", "AvgOffensiveRebDiff","TourneySeed"]], 
                                                            on=["Season", "League", "TeamID"],
                                                            how="left").merge(team_season_agg[["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "AvgTurnoverDiff", "Avg3ptDiff", "AvgAssistDiff", "AvgDefensiveDiff", "AvgOffensiveRebDiff","TourneySeed"]].rename(
    columns = {"TeamID": "OppTeamID", 
               "WinPercentage" : "OppWinPercentage", 
               "MedianScoreDiff": "OppMedianScoreDiff", 
               "TourneySeed" : "OppTourneySeed",
               "AvgTurnoverDiff": "OppAvgTurnoverDiff", 
               "Avg3ptDiff": "OppAvg3ptDiff", 
               "AvgAssistDiff" : "OppAvgAssistDiff", 
               "AvgDefensiveDiff" : "OppAvgDefensiveDiff", 
               "AvgOffensiveRebDiff" : "OppAvgOffensiveRebDiff"}), on=["Season", "League", "OppTeamID"],)
df_historic_tourney_features.head()



,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,AvgOffensiveRebDiff,TourneySeed,OppWinPercentage,OppMedianScoreDiff,OppAvgTurnoverDiff,OppAvg3ptDiff,OppAvgAssistDiff,OppAvgDefensiveDiff,OppAvgOffensiveRebDiff,OppTourneySeed
0,2003,Men,1421,1411,134,92,84,12,15,11,...,-0.384615,16.0,1.0,5.5,2.500000,0.067278,2.777778,-2.500000,-1.500000,16.0
1,2003,Men,1112,1436,136,80,51,16,17,7,...,1.840000,1.0,1.0,10.0,0.526316,0.055832,3.684211,1.210526,4.315789,16.0
2,2003,Men,1113,1272,136,84,71,9,12,6,...,2.777778,10.0,1.0,11.0,-1.434783,0.043182,4.869565,1.913043,1.826087,7.0
3,2003,Men,1141,1166,136,79,73,18,21,3,...,-2.434783,11.0,1.0,19.0,-4.413793,0.048050,5.689655,4.724138,-0.137931,6.0
4,2003,Men,1143,1301,136,76,74,13,14,7,...,0.000000,8.0,1.0,11.5,-1.888889,0.093522,5.277778,2.833333,-0.944444,9.0


In [424]:
df_historic_tourney_features

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,AvgOffensiveRebDiff,TourneySeed,OppWinPercentage,OppMedianScoreDiff,OppAvgTurnoverDiff,OppAvg3ptDiff,OppAvgAssistDiff,OppAvgDefensiveDiff,OppAvgOffensiveRebDiff,OppTourneySeed
0,2003,Men,1421,1411,134,92,84,12,15,11,...,-0.384615,16.0,1.0,5.5,2.500000,0.067278,2.777778,-2.500000,-1.500000,16.0
1,2003,Men,1112,1436,136,80,51,16,17,7,...,1.840000,1.0,1.0,10.0,0.526316,0.055832,3.684211,1.210526,4.315789,16.0
2,2003,Men,1113,1272,136,84,71,9,12,6,...,2.777778,10.0,1.0,11.0,-1.434783,0.043182,4.869565,1.913043,1.826087,7.0
3,2003,Men,1141,1166,136,79,73,18,21,3,...,-2.434783,11.0,1.0,19.0,-4.413793,0.048050,5.689655,4.724138,-0.137931,6.0
4,2003,Men,1143,1301,136,76,74,13,14,7,...,0.000000,8.0,1.0,11.5,-1.888889,0.093522,5.277778,2.833333,-0.944444,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,2023,Women,3276,3261,139,42,66,14,6,4,...,0.590909,6.0,1.0,24.5,-2.785714,0.085607,5.250000,3.750000,7.642857,3.0
4280,2023,Women,3428,3261,144,63,66,15,13,8,...,0.520000,2.0,1.0,24.5,-2.785714,0.085607,5.250000,3.750000,7.642857,3.0
4281,2023,Women,3274,3261,146,42,54,18,12,0,...,3.842105,9.0,1.0,24.5,-2.785714,0.085607,5.250000,3.750000,7.642857,3.0
4282,2023,Women,3439,3261,151,72,79,15,7,9,...,2.148148,1.0,1.0,24.5,-2.785714,0.085607,5.250000,3.750000,7.642857,3.0


In [425]:
df_historic_tourney_features["WinPctDiff"] = (df_historic_tourney_features["WinPercentage"] - df_historic_tourney_features["OppWinPercentage"])
df_historic_tourney_features["TourneySeedDiff"] = df_historic_tourney_features["TourneySeed"] - df_historic_tourney_features["OppTourneySeed"]
df_historic_tourney_features["MedianScoreDiffDiff"] = (df_historic_tourney_features["MedianScoreDiff"] - df_historic_tourney_features["OppMedianScoreDiff"])
df_historic_tourney_features["AvgTurnoverDiffDiff"] = (df_historic_tourney_features["AvgTurnoverDiff"] - df_historic_tourney_features["OppAvgTurnoverDiff"])
df_historic_tourney_features["Avg3ptDiffDiff"] = (df_historic_tourney_features["Avg3ptDiff"] - df_historic_tourney_features["OppAvg3ptDiff"])
df_historic_tourney_features["AvgAssistDiffDiff"] = (df_historic_tourney_features["AvgAssistDiff"] - df_historic_tourney_features["OppAvgAssistDiff"])
df_historic_tourney_features["AvgDefensiveDiffDiff"] = (df_historic_tourney_features["AvgDefensiveDiff"] - df_historic_tourney_features["OppAvgDefensiveDiff"])
df_historic_tourney_features["AvgOffensiveRebDiffDiff"] = (df_historic_tourney_features["AvgOffensiveRebDiff"] - df_historic_tourney_features["OppAvgOffensiveRebDiff"])

In [426]:
df_historic_tourney_features.columns

Index(['Season', 'League', 'TeamID', 'OppTeamID', 'DayNum', 'TeamScore',
       'OppScore', 'TeamTO', 'OppTO', 'TeamFGM3', 'TeamFGA3', 'OppFGM3',
       'OppFGA3', 'TeamAst', 'OppAst', 'TeamStl', 'TeamBlk', 'OppStl',
       'OppBlk', 'TeamOR', 'OppOR', 'GameResult', 'Win', 'WinPercentage',
       'MedianScoreDiff', 'AvgTurnoverDiff', 'Avg3ptDiff', 'AvgAssistDiff',
       'AvgDefensiveDiff', 'AvgOffensiveRebDiff', 'TourneySeed',
       'OppWinPercentage', 'OppMedianScoreDiff', 'OppAvgTurnoverDiff',
       'OppAvg3ptDiff', 'OppAvgAssistDiff', 'OppAvgDefensiveDiff',
       'OppAvgOffensiveRebDiff', 'OppTourneySeed', 'WinPctDiff',
       'TourneySeedDiff', 'MedianScoreDiffDiff', 'AvgTurnoverDiffDiff',
       'Avg3ptDiffDiff', 'AvgAssistDiffDiff', 'AvgDefensiveDiffDiff',
       'AvgOffensiveRebDiffDiff'],
      dtype='object')

In [427]:
ROOT_PATH_MEN = "/kaggle/input/ncaa-men-538-team-ratings/"
ROOT_PATH_WOMEN = "/kaggle/input/ncaa-women-538-team-ratings/"

fivethirtyeight_scores = pd.concat(
    [pd.read_csv(ROOT_PATH_MEN + "538ratingsMen.csv").assign(League="Men"),
     pd.read_csv(ROOT_PATH_WOMEN + "538ratingsWomen.csv").assign(League="Women")
    ]).reset_index(drop=True)


In [428]:
fivethirtyeight_scores.head()

,Season,TeamID,TeamName,538rating,League
0,2016,1242,Kansas,94.46,Men
1,2016,1314,North Carolina,93.94,Men
2,2016,1438,Virginia,92.46,Men
3,2016,1277,Michigan State,91.84,Men
4,2016,1328,Oklahoma,89.96,Men


In [429]:
df_historic_tourney_features = df_historic_tourney_features.merge(
    fivethirtyeight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left").dropna(subset=["538rating"])
df_historic_tourney_features = df_historic_tourney_features.merge(fivethirtyeight_scores.drop("TeamName", axis=1).rename(columns = {"TeamID":"OppTeamID"}), on=["Season", "League", "OppTeamID"], how="left", suffixes=("","Opp"))

In [430]:
df_historic_tourney_features

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,WinPctDiff,TourneySeedDiff,MedianScoreDiffDiff,AvgTurnoverDiffDiff,Avg3ptDiffDiff,AvgAssistDiffDiff,AvgDefensiveDiffDiff,AvgOffensiveRebDiffDiff,538rating,538ratingOpp
0,2016,Men,1195,1192,134,96,65,18,9,8,...,0.0,0.0,2.5,2.213235,0.015793,2.584559,1.268382,3.356618,71.41,66.72
1,2016,Men,1455,1435,134,70,50,7,12,7,...,0.0,0.0,4.0,-6.551487,-0.102764,-0.784897,2.139588,4.336384,86.59,85.59
2,2016,Men,1221,1380,135,59,55,14,9,9,...,0.0,0.0,-3.0,-0.248120,-0.051173,1.488722,-2.710526,-0.372180,66.85,67.96
3,2016,Men,1276,1409,135,67,62,8,11,6,...,0.0,0.0,0.0,2.269048,0.062578,1.695238,-1.354762,0.792857,79.57,79.93
4,2016,Men,1114,1345,136,85,83,9,18,9,...,0.0,7.0,-8.0,-5.666667,0.019371,-3.185185,1.925926,-4.827635,78.90,88.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,2023,Women,3276,3261,139,42,66,14,6,4,...,0.0,3.0,-8.0,-0.623377,-0.032963,-0.159091,-3.295455,-7.051948,87.77,96.14
1830,2023,Women,3428,3261,144,63,66,15,13,8,...,0.0,-1.0,-7.5,0.305714,-0.035558,0.550000,-2.710000,-7.122857,92.15,96.14
1831,2023,Women,3274,3261,146,42,54,18,12,0,...,0.0,6.0,-12.5,-0.898496,-0.070649,-1.881579,-0.013158,-3.800752,83.49,96.14
1832,2023,Women,3439,3261,151,72,79,15,7,9,...,0.0,-2.0,-10.5,1.711640,-0.008661,1.527778,-3.527778,-5.494709,95.72,96.14


In [431]:
df_historic_tourney_features["538rating_diff"] = df_historic_tourney_features["538rating"] - df_historic_tourney_features["538ratingOpp"]

In [432]:
df_historic_tourney_features[["Season", "TeamID", "538rating", "538ratingOpp", "538rating_diff"]]

,Season,TeamID,538rating,538ratingOpp,538rating_diff
0,2016,1195,71.41,66.72,4.69
1,2016,1455,86.59,85.59,1.00
2,2016,1221,66.85,67.96,-1.11
3,2016,1276,79.57,79.93,-0.36
4,2016,1114,78.90,88.68,-9.78
...,...,...,...,...,...
1829,2023,3276,87.77,96.14,-8.37
1830,2023,3428,92.15,96.14,-3.99
1831,2023,3274,83.49,96.14,-12.65
1832,2023,3439,95.72,96.14,-0.42


In [433]:
df_historic_tourney_features["BaselinePred"] = (df_historic_tourney_features["TourneySeed"] < df_historic_tourney_features["OppTourneySeed"])
df_historic_tourney_features.loc[df_historic_tourney_features["TourneySeed"] == df_historic_tourney_features["OppTourneySeed"], "BaselinePred"] = (df_historic_tourney_features["WinPercentage"] > df_historic_tourney_features["OppWinPercentage"])

In [434]:
df_historic_tourney_features

,Season,League,TeamID,OppTeamID,DayNum,TeamScore,OppScore,TeamTO,OppTO,TeamFGM3,...,MedianScoreDiffDiff,AvgTurnoverDiffDiff,Avg3ptDiffDiff,AvgAssistDiffDiff,AvgDefensiveDiffDiff,AvgOffensiveRebDiffDiff,538rating,538ratingOpp,538rating_diff,BaselinePred
0,2016,Men,1195,1192,134,96,65,18,9,8,...,2.5,2.213235,0.015793,2.584559,1.268382,3.356618,71.41,66.72,4.69,False
1,2016,Men,1455,1435,134,70,50,7,12,7,...,4.0,-6.551487,-0.102764,-0.784897,2.139588,4.336384,86.59,85.59,1.00,False
2,2016,Men,1221,1380,135,59,55,14,9,9,...,-3.0,-0.248120,-0.051173,1.488722,-2.710526,-0.372180,66.85,67.96,-1.11,False
3,2016,Men,1276,1409,135,67,62,8,11,6,...,0.0,2.269048,0.062578,1.695238,-1.354762,0.792857,79.57,79.93,-0.36,False
4,2016,Men,1114,1345,136,85,83,9,18,9,...,-8.0,-5.666667,0.019371,-3.185185,1.925926,-4.827635,78.90,88.68,-9.78,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,2023,Women,3276,3261,139,42,66,14,6,4,...,-8.0,-0.623377,-0.032963,-0.159091,-3.295455,-7.051948,87.77,96.14,-8.37,False
1830,2023,Women,3428,3261,144,63,66,15,13,8,...,-7.5,0.305714,-0.035558,0.550000,-2.710000,-7.122857,92.15,96.14,-3.99,True
1831,2023,Women,3274,3261,146,42,54,18,12,0,...,-12.5,-0.898496,-0.070649,-1.881579,-0.013158,-3.800752,83.49,96.14,-12.65,False
1832,2023,Women,3439,3261,151,72,79,15,7,9,...,-10.5,1.711640,-0.008661,1.527778,-3.527778,-5.494709,95.72,96.14,-0.42,True


In [435]:
cv_scores_baseline = []
for season in df_historic_tourney_features["Season"].unique():
    pred = df_historic_tourney_features.query("Season == @season")["BaselinePred"].astype("int")
    y = df_historic_tourney_features.query("Season == @season")["Win"]
    score = accuracy_score(y, pred)
    score_ll = log_loss(y, pred)
    cv_scores_baseline.append(score)
    print(f"Holdout season {season} - Accuracy {score: 0.4f} Log Loss {score_ll: 0.4f}")
print(f"Baseline accuracy {np.mean(cv_scores_baseline):0.4f}")
    
    

Holdout season 2016 - Accuracy  0.6808 Log Loss  11.5062
Holdout season 2017 - Accuracy  0.7808 Log Loss  7.9019
Holdout season 2018 - Accuracy  0.7000 Log Loss  10.8131
Holdout season 2019 - Accuracy  0.7423 Log Loss  9.2882
Holdout season 2021 - Accuracy  0.7364 Log Loss  9.4999
Holdout season 2022 - Accuracy  0.6903 Log Loss  11.1628
Holdout season 2023 - Accuracy  0.7015 Log Loss  10.7593
Baseline accuracy 0.7189


In [436]:
from itertools import combinations
import numpy as np
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb
FEATURES = [
    #"WinPercentage",
    "MedianScoreDiff",
    # "TourneySeed",
    #"OppWinPercentage",
    "OppMedianScoreDiff",
    # "OppTourneySeed",
    #"WinPctDiff",
    #"TourneySeedDiff",
    #"538rating",
    #"538ratingOpp",
    #538rating_diff",
    #'AvgTurnoverDiffDiff',
    'Avg3ptDiffDiff',
    #'AvgAssistDiffDiff',
    'AvgDefensiveDiffDiff',
    #'AvgOffensiveRebDiffDiff' 
]

TARGET = "Win"


X = df_historic_tourney_features[FEATURES]
y = df_historic_tourney_features[TARGET]
groups = df_historic_tourney_features["Season"]
seasons = df_historic_tourney_features["Season"].unique()


gkf = GroupKFold(n_splits=df_historic_tourney_features["Season"].nunique())
cv_results = []
models = []

season_idx = 0
for train_index, test_index in gkf.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    model = xgb.XGBRegressor(
        eval_metric="logloss",
        n_estimators=1_000,
        learning_rate=0.001,
    )
    holdout_season = seasons[season_idx]
    print(f"Holdout Season: {holdout_season}")
    # Train the model
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

    # Predict on the test set
    y_pred = model.predict(X_test)
    score_ll = log_loss(y_test, y_pred)
    y_pred = y_pred > 0.5
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    cv_results.append(accuracy)
    season_idx += 1
    print(f"Season {holdout_season}: {accuracy} {score_ll}")
    models.append(model)

print("Average CV Accuracy:", np.mean(cv_results))

Holdout Season: 2016
[0]	validation_0-logloss:0.69303
[100]	validation_0-logloss:0.68214
[200]	validation_0-logloss:0.67399
[300]	validation_0-logloss:0.66755
[400]	validation_0-logloss:0.66265
[500]	validation_0-logloss:0.65941
[600]	validation_0-logloss:0.65606
[700]	validation_0-logloss:0.65377
[800]	validation_0-logloss:0.65241
[900]	validation_0-logloss:0.65189
[999]	validation_0-logloss:0.65278
Season 2016: 0.5970149253731343 0.6527827981184464
Holdout Season: 2017
[0]	validation_0-logloss:0.69301
[100]	validation_0-logloss:0.68156
[200]	validation_0-logloss:0.67255
[300]	validation_0-logloss:0.66639
[400]	validation_0-logloss:0.66059
[500]	validation_0-logloss:0.65464
[600]	validation_0-logloss:0.64998
[700]	validation_0-logloss:0.64605
[800]	validation_0-logloss:0.64373
[900]	validation_0-logloss:0.64152
[999]	validation_0-logloss:0.63955
Season 2017: 0.6492537313432836 0.6395512500004735
Holdout Season: 2018
[0]	validation_0-logloss:0.69305
[100]	validation_0-logloss:0.68303
[

In [437]:
TEST_SEASON = 2024  # Change to 2024 when it comes out!

seeds_2024 = pd.read_csv(DATA_PATH + "2024_tourney_seeds.csv")

seeds_2024["TourneySeed"] = (
    seeds_2024["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)
seeds_2024["Tournament"] = seeds_2024["Tournament"].replace({"M": "Men", "W": "Women"})
seeds_2024

,Tournament,Seed,TeamID,TourneySeed
0,Men,W01,1163,1
1,Men,W02,1235,2
2,Men,W03,1228,3
3,Men,W04,1120,4
4,Men,W05,1361,5
...,...,...,...,...
123,Women,Z12,3162,12
124,Women,Z13,3267,13
125,Women,Z14,3238,14
126,Women,Z15,3263,15


In [438]:
tourney_pairs = (
    seeds_2024.merge(seeds_2024, on=["Tournament"], suffixes=("", "Opp"))
    .assign(Season=TEST_SEASON)
    .query("TeamID != TeamIDOpp")
    .rename(columns={"Tournament": "League"})
)

tourney_pairs = (
    tourney_pairs.merge(
        team_season_agg[
            ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "AvgTurnoverDiff", "Avg3ptDiff", "AvgAssistDiff", "AvgDefensiveDiff", "AvgOffensiveRebDiff","TourneySeed"]
        ],
        on=["Season", "League", "TeamID"],
        how="left",
    )
    .merge(
        team_season_agg[
            ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "AvgTurnoverDiff", "Avg3ptDiff", "AvgAssistDiff", "AvgDefensiveDiff", "AvgOffensiveRebDiff","TourneySeed"]
        ].rename(
            columns={
                "TeamID": "TeamIDOpp",
                "WinPercentage": "OppWinPercentage",
                "MedianScoreDiff": "OppMedianScoreDiff",
                "Avg3ptDiff":"OppAvg3ptDiff",
                "AvgDefensiveDiff": "OppAvgDefensiveDiff"
            }
        ),
        on=["Season", "League", "TeamIDOpp"],
    )
    .reset_index(drop=True)
)

tourney_pairs["OppTourneySeed"] = (
    tourney_pairs["SeedOpp"]
    .str.replace("a", "")
    .str.replace("b", "")
    .str[1:]
    .astype("int")
)

In [439]:
tourney_pairs

,League,Seed,TeamID,TourneySeed_x,SeedOpp,TeamIDOpp,TourneySeedOpp,Season,WinPercentage,MedianScoreDiff,...,TourneySeed_y,OppWinPercentage,OppMedianScoreDiff,AvgTurnoverDiff_y,OppAvg3ptDiff,AvgAssistDiff_y,OppAvgDefensiveDiff,AvgOffensiveRebDiff_y,TourneySeed,OppTourneySeed
0,Men,W01,1163,1,W02,1235,2,2024,1.0,16.0,...,1.0,1.0,14.0,-8.333333,0.053282,5.407407,5.222222,0.666667,2.0,2
1,Men,W03,1228,3,W02,1235,2,2024,1.0,12.0,...,3.0,1.0,14.0,-8.333333,0.053282,5.407407,5.222222,0.666667,2.0,2
2,Men,W04,1120,4,W02,1235,2,2024,1.0,20.0,...,4.0,1.0,14.0,-8.333333,0.053282,5.407407,5.222222,0.666667,2.0,2
3,Men,W05,1361,5,W02,1235,2,2024,1.0,12.0,...,5.0,1.0,14.0,-8.333333,0.053282,5.407407,5.222222,0.666667,2.0,2
4,Men,W06,1140,6,W02,1235,2,2024,1.0,14.0,...,6.0,1.0,14.0,-8.333333,0.053282,5.407407,5.222222,0.666667,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,Women,Z12,3162,12,W01,3376,1,2024,1.0,18.0,...,12.0,1.0,26.0,-1.656250,0.132762,8.812500,7.000000,3.156250,1.0,1
8060,Women,Z13,3267,13,W01,3376,1,2024,1.0,12.0,...,13.0,1.0,26.0,-1.656250,0.132762,8.812500,7.000000,3.156250,1.0,1
8061,Women,Z14,3238,14,W01,3376,1,2024,1.0,16.0,...,14.0,1.0,26.0,-1.656250,0.132762,8.812500,7.000000,3.156250,1.0,1
8062,Women,Z15,3263,15,W01,3376,1,2024,1.0,12.0,...,15.0,1.0,26.0,-1.656250,0.132762,8.812500,7.000000,3.156250,1.0,1


In [440]:
tourney_pairs.head()
tourney_pairs.to_csv("tourneyPairs.csv")

In [441]:
tourney_pairs = tourney_pairs.merge(
    fivethirtyeight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left",
)

tourney_pairs = tourney_pairs.merge(
    fivethirtyeight_scores.drop("TeamName", axis=1).rename(
        columns={"TeamID": "TeamIDOpp"}
    ),
    on=["Season", "League", "TeamIDOpp"],
    how="left",
    suffixes=("", "Opp"),
)

# Diff features
tourney_pairs["538rating_diff"] = (
    tourney_pairs["538rating"] - tourney_pairs["538ratingOpp"]
)

tourney_pairs["BaselinePred"] = (
    tourney_pairs["TourneySeed"] < tourney_pairs["OppTourneySeed"]
)
tourney_pairs.loc[
    tourney_pairs["TourneySeed"] == tourney_pairs["OppTourneySeed"],
    "BaselinePred",
] = (
    tourney_pairs["WinPercentage"] > tourney_pairs["OppWinPercentage"]
)

tourney_pairs["WinPctDiff"] = (
    tourney_pairs["WinPercentage"] - tourney_pairs["OppWinPercentage"]
)
tourney_pairs["Avg3ptDiffDiff"] = (
    tourney_pairs["Avg3ptDiff"] - tourney_pairs["OppAvg3ptDiff"]
)
tourney_pairs["AvgDefensiveDiffDiff"] = (
    tourney_pairs["AvgDefensiveDiff"] - tourney_pairs["OppAvgDefensiveDiff"]
)

tourney_pairs["TourneySeedDiff"] = (
    tourney_pairs["TourneySeed"] - tourney_pairs["OppTourneySeed"]
)

tourney_pairs["MedianScoreDiffDiff"] = (
    tourney_pairs["MedianScoreDiff"] - tourney_pairs["OppMedianScoreDiff"]
)

In [442]:
tourney_pairs.head()

,League,Seed,TeamID,TourneySeed_x,SeedOpp,TeamIDOpp,TourneySeedOpp,Season,WinPercentage,MedianScoreDiff,...,OppTourneySeed,538rating,538ratingOpp,538rating_diff,BaselinePred,WinPctDiff,Avg3ptDiffDiff,AvgDefensiveDiffDiff,TourneySeedDiff,MedianScoreDiffDiff
0,Men,W01,1163,1,W02,1235,2,2024,1.0,16.0,...,2,NaN,NaN,NaN,False,0.0,0.028912,-1.189964,0.0,2.0
1,Men,W03,1228,3,W02,1235,2,2024,1.0,12.0,...,2,NaN,NaN,NaN,False,0.0,-0.002916,-5.568376,0.0,-2.0
2,Men,W04,1120,4,W02,1235,2,2024,1.0,20.0,...,2,NaN,NaN,NaN,False,0.0,0.052364,-0.629630,0.0,6.0
3,Men,W05,1361,5,W02,1235,2,2024,1.0,12.0,...,2,NaN,NaN,NaN,False,0.0,-0.010559,-1.676768,0.0,-2.0
4,Men,W06,1140,6,W02,1235,2,2024,1.0,14.0,...,2,NaN,NaN,NaN,False,0.0,0.055717,-4.048309,0.0,0.0


In [444]:
for i, model in enumerate(models):
    tourney_pairs[f"pred_model{i}"] = model.predict(tourney_pairs[FEATURES])

In [445]:
tourney_pairs

,League,Seed,TeamID,TourneySeed_x,SeedOpp,TeamIDOpp,TourneySeedOpp,Season,WinPercentage,MedianScoreDiff,...,AvgDefensiveDiffDiff,TourneySeedDiff,MedianScoreDiffDiff,pred_model0,pred_model1,pred_model2,pred_model3,pred_model4,pred_model5,pred_model6
0,Men,W01,1163,1,W02,1235,2,2024,1.0,16.0,...,-1.189964,0.0,2.0,0.521598,0.514399,0.586373,0.551478,0.499642,0.473205,0.514254
1,Men,W03,1228,3,W02,1235,2,2024,1.0,12.0,...,-5.568376,0.0,-2.0,0.405325,0.336309,0.389108,0.399198,0.378917,0.281493,0.345119
2,Men,W04,1120,4,W02,1235,2,2024,1.0,20.0,...,-0.629630,0.0,6.0,0.559811,0.559952,0.517447,0.564430,0.565475,0.663099,0.580765
3,Men,W05,1361,5,W02,1235,2,2024,1.0,12.0,...,-1.676768,0.0,-2.0,0.372733,0.402425,0.400146,0.405219,0.381190,0.351667,0.393151
4,Men,W06,1140,6,W02,1235,2,2024,1.0,14.0,...,-4.048309,0.0,0.0,0.463475,0.420979,0.458435,0.472319,0.555608,0.494597,0.507776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8059,Women,Z12,3162,12,W01,3376,1,2024,1.0,18.0,...,-5.391304,0.0,-8.0,0.345613,0.345689,0.497923,0.297795,0.475519,0.349999,0.370589
8060,Women,Z13,3267,13,W01,3376,1,2024,1.0,12.0,...,-2.500000,0.0,-14.0,0.246091,0.234829,0.327109,0.220723,0.268690,0.240026,0.239674
8061,Women,Z14,3238,14,W01,3376,1,2024,1.0,16.0,...,-4.304348,0.0,-10.0,0.376710,0.299663,0.510974,0.342571,0.314710,0.374495,0.396542
8062,Women,Z15,3263,15,W01,3376,1,2024,1.0,12.0,...,-6.166667,0.0,-14.0,0.505504,0.382924,0.527003,0.618404,0.575235,0.343407,0.439111


In [447]:
tourney_pairs["Pred"] = tourney_pairs[
    [f for f in tourney_pairs.columns if "model" in f]
].mean(axis=1)

tourney_pairs["ID"] = (
    tourney_pairs["Season"].astype("str")
    + "_"
    + tourney_pairs["TeamID"].astype("str")
    + "_"
    + tourney_pairs["TeamIDOpp"].astype("str")
)

preds = tourney_pairs.copy()

In [448]:
tourney_pairs.head()

,League,Seed,TeamID,TourneySeed_x,SeedOpp,TeamIDOpp,TourneySeedOpp,Season,WinPercentage,MedianScoreDiff,...,MedianScoreDiffDiff,pred_model0,pred_model1,pred_model2,pred_model3,pred_model4,pred_model5,pred_model6,Pred,ID
0,Men,W01,1163,1,W02,1235,2,2024,1.0,16.0,...,2.0,0.521598,0.514399,0.586373,0.551478,0.499642,0.473205,0.514254,0.522993,2024_1163_1235
1,Men,W03,1228,3,W02,1235,2,2024,1.0,12.0,...,-2.0,0.405325,0.336309,0.389108,0.399198,0.378917,0.281493,0.345119,0.362210,2024_1228_1235
2,Men,W04,1120,4,W02,1235,2,2024,1.0,20.0,...,6.0,0.559811,0.559952,0.517447,0.564430,0.565475,0.663099,0.580765,0.572997,2024_1120_1235
3,Men,W05,1361,5,W02,1235,2,2024,1.0,12.0,...,-2.0,0.372733,0.402425,0.400146,0.405219,0.381190,0.351667,0.393151,0.386647,2024_1361_1235
4,Men,W06,1140,6,W02,1235,2,2024,1.0,14.0,...,0.0,0.463475,0.420979,0.458435,0.472319,0.555608,0.494597,0.507776,0.481884,2024_1140_1235


In [449]:
from tqdm import tqdm

# Load and filter data
round_slots = pd.read_csv(
    "/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv"
)
round_slots = round_slots[round_slots["Season"] == 2023]
round_slots = round_slots[
    round_slots["Slot"].str.contains("R")
]  # Filter out First Four

seeds = pd.read_csv(
    "/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv"
)
seeds_m = seeds[seeds["Tournament"] == "M"]
seeds_w = seeds[seeds["Tournament"] == "W"]

preds["ID"] = preds["ID"].str.split("_")

In [450]:
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["Pred"]):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    winners = []
    slots = []

    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning
        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1
result_m = run_simulation(
    brackets=n_brackets, seeds=seeds_m, preds=preds, round_slots=round_slots, sim=False
)
result_m["Tournament"] = "M"
result_w = run_simulation(
    brackets=n_brackets, seeds=seeds_w, preds=preds, round_slots=round_slots, sim=False
)
result_w["Tournament"] = "W"
submission = pd.concat([result_m, result_w])
submission = submission.reset_index(drop=True)
submission.index.names = ["RowId"]
submission = submission.reset_index()

100%|██████████| 1/1 [00:00<00:00, 297.17it/s]


In [451]:
submission_with_names = submission.rename(columns={"Team": "Seed"}).merge(
    seeds, on=["Seed", "Tournament"], how="left"
)

teams = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MTeams.csv").assign(Tournament="M"),
        pd.read_csv(DATA_PATH + "WTeams.csv").assign(Tournament="W"),
    ]
)

submission_with_names = submission_with_names.merge(
    teams[["Tournament", "TeamID", "TeamName"]], how="left"
)

In [452]:
submission_with_names.to_csv("submission_with_names.csv")

In [453]:
submission_with_names

,index,Bracket,Slot,Seed,Tournament,TeamID,TeamName
0,0,1,R1W1,W01,M,1163,Connecticut
1,1,1,R1W2,W02,M,1235,Iowa St
2,2,1,R1W3,W14,M,1287,Morehead St
3,3,1,R1W4,W04,M,1120,Auburn
4,4,1,R1W5,W12,M,1412,UAB
...,...,...,...,...,...,...,...
121,121,1,R4Y1,Y08,W,3452,West Virginia
122,122,1,R4Z1,Z03,W,3163,Connecticut
123,123,1,R5WX,W01,W,3376,South Carolina
124,124,1,R5YZ,Z03,W,3163,Connecticut
